In [15]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [17]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 195 images belonging to 2 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 58 images belonging to 2 classes.


In [19]:
cnn = tf.keras.models.Sequential()

In [20]:
# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


# Step 3 - Flattening
cnn.add(tf.keras.layers.GlobalAveragePooling2D())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="model.h5", save_best_only=True)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=10)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

clbs = [reduce_lr_callback , early_stopping_callback ]

cnn.summary()

# Training the CNN on the Training set and evaluating it on the Test set

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                  

In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 35)


Epoch 1/35
7/7 [==============================] - 3s 303ms/step - loss: 0.6564 - accuracy: 0.6462 - val_loss: 0.7370 - val_accuracy: 0.5000
Epoch 2/35
7/7 [==============================] - 2s 244ms/step - loss: 0.6325 - accuracy: 0.6462 - val_loss: 0.7275 - val_accuracy: 0.5000
Epoch 3/35
7/7 [==============================] - 2s 213ms/step - loss: 0.6211 - accuracy: 0.6462 - val_loss: 0.7142 - val_accuracy: 0.5000
Epoch 4/35
7/7 [==============================] - 2s 210ms/step - loss: 0.6201 - accuracy: 0.6462 - val_loss: 0.7292 - val_accuracy: 0.5000
Epoch 5/35
7/7 [==============================] - 1s 199ms/step - loss: 0.6121 - accuracy: 0.6462 - val_loss: 0.6786 - val_accuracy: 0.5000
Epoch 6/35
7/7 [==============================] - 2s 206ms/step - loss: 0.6230 - accuracy: 0.6462 - val_loss: 0.6704 - val_accuracy: 0.5862
Epoch 7/35
7/7 [==============================] - 2s 217ms/step - loss: 0.6115 - accuracy: 0.6462 - val_loss: 0.7108 - val_accuracy: 0.5000
Epoch 8/35
7/7 [====

In [25]:
# Part 4 - Making a single prediction

import numpy as np
# from keras.preprocessing import image
import keras.utils as image
test_image = image.load_img('dataset/single_prediction/yes_or_no.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Yes, Brain tumor is present.'
else:
    prediction = 'No, Brain Tumore is absent.'
print(prediction)

1/1 [==============================] - 0s 67ms/step
Yes, Brain tumor is present.


In [28]:
import pickle
filename = 'brain-tumor-mri-model.pkl'
pickle.dump(cnn, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://ff025077-57b2-4212-85df-6d0b9c368467/assets


INFO:tensorflow:Assets written to: ram://ff025077-57b2-4212-85df-6d0b9c368467/assets
